### secondary labels memo

In [1]:
import pandas as pd
import ast

In [2]:
# Read csv
train = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/train_metadata.csv")
train["secondary_labels"] = train["secondary_labels"].apply(ast.literal_eval)
sample_submission = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/sample_submission.csv")
# Retrieve target
birds = sample_submission.columns[1:]
# Label encoding
train = pd.concat([train, pd.get_dummies(train["primary_label"])], axis=1)
# Fix order
new_columns = list(train.columns.difference(birds)) + list(birds)
train = train.reindex(columns=new_columns)

In [3]:
train_copy = train.copy()
# very stupid code...
for idx, each_secondary_labels in enumerate(train["secondary_labels"]):
    for secondary_label in each_secondary_labels:
        for bird in birds:
            if secondary_label == bird:
                train_copy.loc[idx, bird] = 1

In [8]:
train_copy.iloc[:, -264:][5]

KeyError: 5

In [311]:
for idx, i in enumerate(train_copy.iloc[17]):
    print(train_copy.columns[idx], i)

author David Moyer
common_name African Black-headed Oriole
filename abhori1/XC127317.ogg
latitude -5.7214
license Creative Commons Attribution-NonCommercial-ShareAlike 3.0
longitude 37.9942
primary_label abhori1
rating 3.5
scientific_name Oriolus larvatus
secondary_labels ['blbpuf2', 'fotdro5', 'reedov1']
type ['song']
url https://www.xeno-canto.org/127317
abethr1 0
abhori1 1
abythr1 0
afbfly1 0
afdfly1 0
afecuc1 0
affeag1 0
afgfly1 0
afghor1 0
afmdov1 0
afpfly1 0
afpkin1 0
afpwag1 0
afrgos1 0
afrgrp1 0
afrjac1 0
afrthr1 0
amesun2 0
augbuz1 0
bagwea1 0
barswa 0
bawhor2 0
bawman1 0
bcbeat1 0
beasun2 0
bkctch1 0
bkfruw1 0
blacra1 0
blacuc1 0
blakit1 0
blaplo1 0
blbpuf2 1
blcapa2 0
blfbus1 0
blhgon1 0
blhher1 0
blksaw1 0
blnmou1 0
blnwea1 0
bltapa1 0
bltbar1 0
bltori1 0
blwlap1 0
brcale1 0
brcsta1 0
brctch1 0
brcwea1 0
brican1 0
brobab1 0
broman1 0
brosun1 0
brrwhe3 0
brtcha1 0
brubru1 0
brwwar1 0
bswdov1 0
btweye2 0
bubwar2 0
butapa1 0
cabgre1 0
carcha1 0
carwoo1 0
categr 0
ccbeat1 0
che

### pretraining pipeline memo 

In [28]:
for i, code in enumerate(xeno.ebird_code):
    if code[0] == "n":
        print(i)
        break

14685


In [4]:
import pandas as pd

In [216]:
xeno_am = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/xenocanto/train_extended.csv")[:14684] # 0 ~ 14864 -> A-M, 14865 ~ -> N-Z
xeno_nz = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/xenocanto/train_extended.csv")[14685:]
bird2020 = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/bird2020/train.csv")
bird2021 = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/bird2021/train_metadata.csv")
bird2022 = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/bird2022/train_metadata.csv")
bird2023 = pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/train_metadata.csv")

In [217]:
print(pd.read_csv("/Users/moritake/data_science/kaggle/birdclef-2023/data/input/xenocanto/train_extended.csv").shape)
print(bird2020.shape)
print(bird2021.shape)
print(bird2022.shape)
print(bird2023.shape)

(23784, 29)
(21375, 35)
(62874, 14)
(14852, 13)
(16941, 12)


In [218]:
xeno_am["birdclef"] = "xeno"
xeno_nz["birdclef"] = "xeno"
bird2020["birdclef"] = "2020"
bird2021["birdclef"] = "2021"
bird2022["birdclef"] = "2022"

xeno_am["primary_label"] = xeno_am["ebird_code"]
xeno_nz["primary_label"] = xeno_nz["ebird_code"]
bird2020["primary_label"] = bird2020["ebird_code"]

xeno_am["filepath"] = xeno_am["primary_label"] + "/" + xeno_am["filename"]
xeno_nz["filepath"] = xeno_nz["primary_label"] + "/" + xeno_nz["filename"]
bird2020["filepath"] = bird2020["primary_label"] + "/" + bird2020["filename"]
bird2021["filepath"] = bird2021["primary_label"] + "/" + bird2021["filename"]
bird2022["filepath"] = bird2022["filename"]

In [219]:
bird2023["filepath"] = bird2023["filename"]
bird2023["filename"] = bird2023["filepath"].map(lambda x: x.split("/")[-1].split(".")[0])

In [220]:
pre_df = pd.concat([xeno_am, xeno_nz, bird2020, bird2021, bird2022], axis=0).reset_index(drop=True)
print(pre_df.shape)
pre_df["filepath"] = pre_df["filepath"].map(lambda x: x.split(".")[0])
pre_df["filename"] = pre_df["filepath"].map(lambda x: x.split("/")[-1])
nodup_idx = pre_df[["filename", "primary_label", "author"]].drop_duplicates().index
pre_df = pre_df.loc[nodup_idx].reset_index(drop=True)

(122884, 39)


In [222]:
pre_df.shape

(84258, 39)

In [223]:
pre_df = pre_df[~pre_df["filename"].isin(bird2023["filename"])].reset_index(drop=True)
pre_df = pre_df[["filepath", "filename", "primary_label", "secondary_labels", "birdclef"]]

In [224]:
pre_df.shape

(83238, 5)

In [225]:
pre_df

,filepath,filename,primary_label,secondary_labels,birdclef
0,aldfly/XC554809,XC554809,aldfly,[],xeno
1,aldfly/XC552408,XC552408,aldfly,[],xeno
2,aldfly/XC544552,XC544552,aldfly,[],xeno
3,aldfly/XC544551,XC544551,aldfly,[],xeno
4,aldfly/XC544550,XC544550,aldfly,[],xeno
...,...,...,...,...,...
83233,zebdov/XC629769,XC629769,zebdov,[],2022
83234,zebdov/XC642415,XC642415,zebdov,[],2022
83235,zebdov/XC665873,XC665873,zebdov,[],2022
83236,zebdov/XC666194,XC666194,zebdov,[],2022


In [175]:
xeno_am["xc_id"]

0        554809
1        552408
2        544552
3        544551
4        544550
          ...  
14679    559693
14680    534461
14681    567994
14682    522348
14683    326248
Name: xc_id, Length: 14684, dtype: int64

In [11]:
bird2022.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,time,url,filename
0,afrsil1,[],"['call', 'flight call']",12.3910,-1.4930,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,2.5,08:00,https://www.xeno-canto.org/125458,afrsil1/XC125458.ogg
1,afrsil1,"['houspa', 'redava', 'zebdov']",['call'],19.8801,-155.7254,Euodice cantans,African Silverbill,Dan Lane,Creative Commons Attribution-NonCommercial-Sha...,3.5,08:30,https://www.xeno-canto.org/175522,afrsil1/XC175522.ogg
2,afrsil1,[],"['call', 'song']",16.2901,-16.0321,Euodice cantans,African Silverbill,Bram Piot,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:30,https://www.xeno-canto.org/177993,afrsil1/XC177993.ogg
3,afrsil1,[],"['alarm call', 'call']",17.0922,54.2958,Euodice cantans,African Silverbill,Oscar Campbell,Creative Commons Attribution-NonCommercial-Sha...,4.0,11:00,https://www.xeno-canto.org/205893,afrsil1/XC205893.ogg
4,afrsil1,[],['flight call'],21.4581,-157.7252,Euodice cantans,African Silverbill,Ross Gallardy,Creative Commons Attribution-NonCommercial-Sha...,3.0,16:30,https://www.xeno-canto.org/207431,afrsil1/XC207431.ogg


In [13]:
bird2023.head()

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,abethr1,[],['song'],4.3906,38.2788,Turdus tephronotus,African Bare-eyed Thrush,Rolf A. de By,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/128013,abethr1/XC128013.ogg
1,abethr1,[],['call'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363501,abethr1/XC363501.ogg
2,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,3.5,https://www.xeno-canto.org/363502,abethr1/XC363502.ogg
3,abethr1,[],['song'],-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/363503,abethr1/XC363503.ogg
4,abethr1,[],"['call', 'song']",-2.9524,38.2921,Turdus tephronotus,African Bare-eyed Thrush,James Bradley,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://www.xeno-canto.org/363504,abethr1/XC363504.ogg
